# Coleta de Dados do Sentinel-2 usando Google Earth Engine

Este notebook demonstra como coletar dados do Sentinel-2 para áreas de interesse definidas em arquivos KML.

In [2]:
import sys
sys.path.append('..')

import ee
from pathlib import Path
from src.data.gee_collector import SentinelCollector
from src.data.dataset_manager import DatasetManager

## 1. Configurar o Ambiente

Primeiro, vamos configurar os diretórios e inicializar os componentes necessários:

In [3]:
# Configurar diretórios
data_dir = Path('../data')
kml_dir = data_dir / 'kmls'  # Diretório único para todos os KMLs
output_dir = data_dir / 'tiffs'

# Criar diretórios se não existirem
kml_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# Inicializar o coletor com o diretório dos KMLs
collector = SentinelCollector(
    train_val_dir=str(kml_dir)
)

## 2. Carregar e Dividir Regiões

O SentinelCollector usa internamente o DatasetManager para gerenciar os KMLs:

In [4]:
# Carregar e dividir regiões usando o dataset_manager interno do collector
train_regions, val_regions = collector.dataset_manager.load_train_val_regions(val_split=0.2)

print(f"Número de regiões para treino: {len(train_regions)}")
print(f"Número de regiões para validação: {len(val_regions)}")

Número de regiões para treino: 4
Número de regiões para validação: 1


## 3. Coletar Imagens do Sentinel-2

Agora vamos coletar as imagens para as regiões de treino e validação:

In [5]:
# Parâmetros de coleta
collection_params = {
    'years_back': 5,  # Últimos 2 anos de dados
    'max_cloud_cover': 20  # Máximo de 20% de cobertura de nuvens
}

# Coletar imagens
train_collection = collector.get_sentinel_collection(
    regions=train_regions,
    **collection_params
)

val_collection = collector.get_sentinel_collection(
    regions=val_regions,
    **collection_params
)

## 4. Exportar para GeoTIFF

Vamos exportar as imagens coletadas para arquivos GeoTIFF:

In [6]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'patch_size': 64,  # Tamanho do patch em pixels
    'bands': ['B8', 'B4', 'B11']  # Bandas relevantes para vegetação
}

# Exportar dados de treino
collector.export_to_drive(
    collection=train_collection,
    regions=train_regions,
    output_prefix='treino',
    **export_params
)

# Exportar dados de validação
collector.export_to_drive(
    collection=val_collection,
    regions=val_regions,
    output_prefix='validacao',
    **export_params
)

Started export task: treino_sentinel2
You can monitor the task status at: https://code.earthengine.google.com/tasks
Started export task: validacao_sentinel2
You can monitor the task status at: https://code.earthengine.google.com/tasks


## 5. Monitorar Exportação e Próximos Passos

### Monitoramento
As tarefas de exportação serão executadas no Google Earth Engine. Você pode monitorar o progresso em:
https://code.earthengine.google.com/tasks

### Organização dos KMLs
- Coloque todos os seus arquivos KML no diretório `kmls`
- O SentinelCollector (através do DatasetManager) irá automaticamente:
  - Carregar todos os KMLs do diretório
  - Dividir as regiões em treino/validação
  - Manter a consistência entre as divisões

### Após a Exportação
1. Baixe os arquivos GeoTIFF do Google Drive
2. Mova-os para o diretório `tiffs` do projeto
3. Use o TiffLoader para preparar os dados para treinamento

### Importante
Certifique-se de que você tem:
- Acesso configurado ao Google Earth Engine
- Espaço suficiente no Google Drive
- KMLs bem definidos para suas áreas de interesse